## Imports

In [11]:
import itertools
import mdptoolbox.example
import numpy as np

## Variable Initialization

In [12]:
items = 3
factory_size = 4
#we can also set the probability for each item to be stored or unstored
whites = 0.9
reds = 0.05
blues = 0.05
trans_probas = [whites/2, reds/2, blues/2, whites/2, reds/2, blues/2]

not_possible_reward = -10


# encoding: x = between(0,actions-1): store on field x; x = between(actions, actions*2-1): restore on field x - actions
actions = range(factory_size * 2)

# encoding: x = between(0,items-1): store item x; x = between(items, items*2-1): restore item x - items
tasks = range(items * 2)

field_combs = list(itertools.product(range(items + 1), repeat=factory_size))
# encoding: ((0,0,0,0),0) until ((3,3,3,3),5)
states = list(itertools.product(range(items * 2), field_combs, repeat=1))


## initializing transition probability matrix and reward matrix

Before we can create and fill the transition probability matrix and the reward matrix,
we have to create some helper functions.

The function "is_transition_possible(action, s, s_prime)" checks, whether we can transition from state s to state s_prime
by performing action action

In [13]:
def is_transition_possible(action, s, s_prime):
        task = states[s][0] + 1
        if action < factory_size:
            stored_fields = states[s_prime][1]
            unstored_fields = states[s][1]
        else:
            action -= factory_size
            task -= items
            stored_fields = states[s][1]
            unstored_fields = states[s_prime][1]
        diff = np.subtract(stored_fields, unstored_fields)
        if len(np.nonzero(diff)[0]) != 1 or unstored_fields[action] != 0 or stored_fields[action] != task or \
                stored_fields[action] == unstored_fields[action]:
            return False
        return True

we can now create the transition and reward matrices

In [14]:
transitions = np.zeros(shape=(len(actions), len(states), len(states)))
rewards = np.full((len(states), len(actions)), not_possible_reward)
for action in range(len(transitions)):
    for s in range(len((transitions[action]))):
        reward_found = False
        for s_prime in range(len(transitions[action][s])):
            if is_transition_possible(action, s, s_prime):
                transitions[action][s][s_prime] = trans_probas[states[s_prime][0]]
                reward_found = True
        if reward_found:
            rewards[s][action] = -action
        if np.sum(transitions[action][s]) == 0:
            base_state = s % len(field_combs)
            same_states = np.where(np.asarray(range(len(states))) % len(field_combs) == base_state)
            transitions[action][s][same_states] = trans_probas

## Training the models

In [15]:
value_iter = mdptoolbox.mdp.ValueIteration(transitions, rewards, 1, epsilon=0.001)
value_iter.setVerbose()
value_iter.run()
print("trained for " + str(value_iter.iter) + " iterations")
print("time:  " + str(value_iter.time))
print("mdp.policy")
vi_policy = value_iter.policy
print(value_iter.policy)

  Iteration		V-variation
    1		  10.0
    2		  7.275000000000002
    3		  6.4856250000000015
    4		  5.929546875000007
    5		  5.637715624999995
    6		  5.407748974609383
    7		  5.2791744326172
    8		  5.062333699774186
    9		  4.957008028343033
    10		  4.795280849099775
    11		  4.692295335437642
    12		  4.566898976156345
    13		  4.4707248636631505
    14		  4.36322886194074
    15		  4.272069296747858
    16		  4.177506687421314
    17		  4.092640391047709
    18		  4.00783205827527
    19		  3.9288260268428843
    20		  3.8528067441562968
    21		  3.7831887256046883
    22		  3.7165296197957076
    23		  3.650956930783124
    24		  3.5858875300550324
    25		  3.522636282137782
    26		  3.459974675752477
    27		  3.3984165891236415
    28		  3.3376578500826497
    29		  3.277906461575242
    30		  3.219101238323219
    31		  3.1613014227971235
    32		  3.1044811648223742
    33		  3.048814953609252
    34		  2.9953245840244733
    35		  2.942466003796966
    36		 

We can now train other mdp classes for comparison

In [16]:
value_iter_gs = mdptoolbox.mdp.ValueIterationGS(transitions, rewards, 1, epsilon=0.001)
value_iter_gs.run()
vi_gs_policy = value_iter_gs.policy

rel_value_iter = mdptoolbox.mdp.RelativeValueIteration(transitions, rewards, epsilon=0.001)
rel_value_iter.run()
rel_vi_policy = rel_value_iter.policy

q_learn = mdptoolbox.mdp.QLearning(transitions, rewards, 1)
q_learn.run()
q_learn_policy = q_learn.policy

pol_iter = mdptoolbox.mdp.PolicyIteration(transitions, rewards, 1)
pol_iter.run()
pol_iter_policy = pol_iter.policy

Before we can evaluate the policy, we need to generate some data

In [17]:
data_amount = 1000
data = np.random.choice(range(items*2), p=trans_probas,size=data_amount)

We can now define a function that evaluates the policy with the generated data

In [18]:
def evaluate(policy, data):
    factory = (0, 0, 0, 0)
    reward = 0
    for task in data:
        x = np.array(list(map(lambda x: x==(task, factory), states)))
        curr_state = np.where(x)[0][0]
        action = policy[curr_state]
        reward += rewards[curr_state][action]
        factory = states[np.where(transitions[action][curr_state]== trans_probas[0])[0][0]][1]
    avg_reward = reward/len(data)
    return avg_reward


## Evaluation

We can now evaluate the different trained mdp solving algorithms

In [19]:
print("ValueIteration")
print(evaluate(vi_policy, data))
print("ValueIterationGS")
print(evaluate(vi_gs_policy, data))
print("RelativeValueIteration")
print(evaluate(rel_vi_policy, data))
print("QLearning")
print(evaluate(q_learn_policy, data))
print("PolicyIteration")
print(evaluate(pol_iter_policy, data))

ValueIteration
-4.93
ValueIterationGS
-5.117
RelativeValueIteration
-4.93
QLearning
-9.694
PolicyIteration
-9.974


## Greedy Evaluation

for a final comparison, those algorithms are being compared to the greedy algorithm

In [20]:
factory = [0, 0, 0, 0]
reward = 0
for task in data:
    if task < 3: #store item task
        needed = 0
    else:
        needed = task - 2
    available_space = np.asarray(np.asarray(factory) == needed).nonzero()
    if len(available_space[0]) >= 1:
        index = available_space[0][0]
        if task < 3:
            factory[index] = task + 1
        else:
            factory[index] = 0
        reward -= index
    else:
        reward += not_possible_reward

avg_reward = reward/len(data)
print("Greedy Algorithm")
print(avg_reward)

Greedy Algorithm
-5.908
